
# Regresja liniowa dla ciągów binarnych

W tym notebooku zajmiemy się zadaniem regresji liniowej, w którym wejściem są ciągi binarne
(np. `010011...`), a wyjściem jest wartość pewnej **tajnej funkcji**.



## Opis zadania

Mamy tajną funkcję $f$, której argumentem jest ciąg binarny o stałej długości:

$$
f(\text{ciąg binarny}) =
a_0 \cdot \text{ile(0)} +
a_1 \cdot \text{ile(1)} +
a_{00} \cdot \text{ile(00)} +
a_{01} \cdot \text{ile(01)} +
a_{10} \cdot \text{ile(10)} +
a_{11} \cdot \text{ile(11)} + \text{szum}.
$$

### Co oznaczają składniki?

Dla danego ciągu binarnego:

* `ile(0)` – liczba symboli `0` w ciągu,
* `ile(1)` – liczba symboli `1` w ciągu,
* `ile(00)` – liczba **kolejnych** par `00` w ciągu,
* `ile(01)` – liczba **kolejnych** par `01` w ciągu,
* `ile(10)` – liczba **kolejnych** par `10` w ciągu,
* `ile(11)` – liczba **kolejnych** par `11` w ciągu.

Przy liczeniu, pary **mogą się nachodzić**, tzn. dla ciągu `0101` patrzymy na pary:

* pozycje (1,2): `01`
* pozycje (2,3): `10`
* pozycje (3,4): `01`

czyli dla `0101` mamy:
* `ile(0) = 2`,
* `ile(1) = 2`,
* `ile(01) = 2`,
* `ile(10) = 1`,
* `ile(00) = 0`,
* `ile(11) = 0`.

Twoim zadaniem jest **odtworzenie współczynników** $a_0, a_1, a_{00}, a_{01}, a_{10}, a_{11}$ na podstawie danych treningowych, a następnie wykorzystanie wytrenowanego modelu do
przewidywania wartości funkcji $f$ dla nowych ciągów binarnych.



## Dane wejściowe i wyjściowe

Na repozytorium załączone są 2 pliki w formacie `npz` (zapisane za pomocą `numpy.savez`):

* **`train.npz`** – dane treningowe
  * klucz `"xs"`: tablica typu `numpy.ndarray` o wymiarach `(num_samples, seq_len)`,
    zawierająca ciągi binarne (0/1),
  * klucz `"y"`: tablica o wymiarach `(num_samples,)` zawierająca wartości funkcji $f$
    dla odpowiadających im ciągów.
* **`test.npz`** – dane testowe
  * klucz `"xs"`: tablica o wymiarach `(num_test_samples, seq_len)` z ciągami binarnymi,
  * **brak** wartości $f$ – to właśnie te wartości masz przewidzieć.

### Format zgłoszenia rozwiązania

Rozwiązaniem jest **jeden plik** w formacie `npz`, zapisany np. tak:

```python
np.savez("solution.npz", y=y_pred)
```

gdzie:
* `y_pred` to tablica `numpy.ndarray` z przewidywanymi wartościami funkcji $f$
  dla przykładów z `test.npz`,
* nazwa klucza musi być dokładnie `"y"`.



## Kryterium oceny – błąd MSE

Jakość przewidywań mierzymy za pomocą błędu średniokwadratowego (MSE – *Mean Squared Error*).

Jeśli mamy prawdziwe wartości \\( y_1, \dots, y_n \\) oraz przewidywania modelu
\\( \\hat{y}_1, \dots, \\hat{y}_n \\), to:

\\[
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2.
\\]

Im **mniejsza** wartość MSE, tym lepszy model.

### Punktacja

Punktacja zależy od MSE na zbiorze testowym:

* jeśli \\( \mathrm{MSE} > 1 \\) – dostajesz **0 punktów**,
* jeśli \\( \mathrm{MSE} < 0.2 \\) – dostajesz **1 punkt**,
* jeśli \\( 1 \le \mathrm{MSE} \le 1 \\) – punkty są skalowane **liniowo**.

Zatem możemy zapisać liczbę punktów jako:

$$
\text{punkty}(\mathrm{MSE}) =
\begin{cases}
1, & \text{gdy } \mathrm{MSE} \le 0.2,\\
0, & \text{gdy } \mathrm{MSE} \ge 1,\\
\dfrac{1 - \mathrm{MSE}}{0.8}, & \text{w przeciwnym razie.}
\end{cases}
$$


# Rozwiązanie

## Wczytywanie danych

In [2]:
import numpy as np

# Wczytujemy dane
train = np.load("train.npz")
test = np.load("test.npz")

xs_train = train["xs"]
y_train = train["y"]
xs_test = test["xs"]

print("Wymiary xs_train:", xs_train.shape)
print("Wymiary y_train:", y_train.shape)
print("Wymiary xs_test:", xs_test.shape)

Wymiary xs_train: (2000, 20)
Wymiary y_train: (2000,)
Wymiary xs_test: (500, 20)


## Drobna wizualizacja danych

In [3]:
import pandas as pd

df = pd.DataFrame(xs_train)
df['y'] = y_train
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,y
0,1,1,1,1,1,0,1,0,1,0,...,1,1,1,0,1,1,0,0,0,-23.137494
1,0,1,1,0,1,1,0,1,0,1,...,0,0,1,0,1,1,1,1,0,-23.619056
2,0,0,0,0,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,-9.626057
3,0,0,1,0,0,0,1,1,1,0,...,1,1,1,0,0,1,1,0,1,-13.278507
4,1,0,1,0,0,1,1,0,1,0,...,0,1,1,1,1,0,1,0,1,-20.560727


## Przykładowe rozwiązanie

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Przykładowe rozwiązanie - zwracamy średnią ze zbioru treningowego
mean_result = y_train.mean()
y_train_pred = np.full(len(xs_train), mean_result)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"MSE na zbiorze treningowym: {mse_train:.4f}")

y_test_pred = np.full(len(xs_test), mean_result)

MSE na zbiorze treningowym: 60.4144


In [5]:
# Zapisujemy rozwiązanie
assert y_test_pred.shape == (xs_test.shape[0],), "Nieprawidłowe wymiary zapisywanych wyników!"
np.savez("solution.npz", y=y_test_pred)
print("Zapisano plik 'solution.npz' z wynikami dla zbioru testowego.")

Zapisano plik 'solution.npz' z wynikami dla zbioru testowego.
